# Hubside


## 1) Catégoriser nos sites clients publiés: 
- A partir du contenu des sites
- Généré des mots clefs 
- Catégorisé
- Limitation de la taille context

Aujourd'hui, nous ne sommes pas en mesure de savoir de quoi parle les sites de nos clients, autrement que par les browser à la main. Pour un usage interne, j'ai demandé à l'API de me donner pour un site donné, des tags, une description, et de choisir parmi nos catégories (celles des sites modèles sur hubside.com) laquelle match le mieux. Pour exploiter le résultat via nos services, j'ai demandé d'avoir tout ça sous forme de JSON.

Exemple de prompt:
Give me french tags describing this website hardrunning.hubside.es. Give me a french short description of what the website is about. Choose the best categories for this site between: BOUTIQUE EN LIGNE, SERVICES, MODE & BEAUTÉ, LOISIRS, BLOG, PORTFOLIO & CV, ÉVÈNEMENTS, ART & CULTURE, GASTRONOMIE, ASSOCIATION & INSTITUTION. The result should be in french and in json format, parent keys are tags, description, category
Réponse OpenAI:



In [3]:
import vertexai

PROJECT_ID = "ml-demo-384110"  # @param {type:"string"}
REGION = "us-central1"

vertexai_text_bison ="text-bison@001"

vertexai.init(project=PROJECT_ID, location=REGION)

In [4]:
import logging
logger = logging.getLogger("root")
logger.setLevel(logging.DEBUG)


In [5]:
import pandas as pd

from vertexai.preview.language_models import TextGenerationModel

In [6]:
generation_model = TextGenerationModel.from_pretrained(vertexai_text_bison)

def llm(prompt):
    result = generation_model.predict(
        prompt=prompt,
        max_output_tokens=1024,
        temperature=0,
        top_p=0.1,
        top_k=1,        
    )
    return result

### URL and prompt

In [ ]:
url = "https://hardrunning.hubside.es"

### basic approach

In [ ]:
llm(
 f"""Give me tags describing this website {url}. 
Give me a short description of what the website is about. 
Choose the best categories for this site between: 
BOUTIQUE EN LIGNE, 
SERVICES, MODE & BEAUTÉ, 
LOISIRS, 
BLOG, 
PORTFOLIO & CV, 
ÉVÈNEMENTS, 
ART & CULTURE, 
GASTRONOMIE, 
ASSOCIATION & INSTITUTION. 

The result should be in json format, parent keys are tags, description, category
""")

{
  "tags": [
    "running",
    "sport",
    "training",
    "fitness",
    "health",
    "nutrition",
    "lifestyle"
  ],
  "description": "Hardrunning is a website that provides information about running, training, fitness, health, nutrition and lifestyle. It also offers a forum for users to discuss these topics.",
  "category": "LOISIRS"
}

### Can I trust answer ?

In [ ]:
url = "https://tintin_le_film.hubside.es"

llm(
 f"""Give me tags describing this website {url}. 
Give me a short description of what the website is about. 
Choose the best categories for this site between: 
BOUTIQUE EN LIGNE, 
SERVICES, MODE & BEAUTÉ, 
LOISIRS, 
BLOG, 
PORTFOLIO & CV, 
ÉVÈNEMENTS, 
ART & CULTURE, 
GASTRONOMIE, 
ASSOCIATION & INSTITUTION. 

The result should be in json format, parent keys are tags, description, category
""")

{
  "tags": [
    "tintin",
    "film",
    "hubside"
  ],
  "description": "Tintin le film is a 2011 French-Belgian animated adventure film directed by Steven Spielberg and produced by Peter Jackson. It is the third and final film in the Tintin film series, based on The Adventures of Tintin, the comic book series by Belgian cartoonist Hergé. The film stars Jamie Bell as Tintin, Andy Serkis as Captain Haddock, Daniel Craig as Ivan Ivanovitch Sakharine, Simon Pegg as Thomson and Thompson, Nick Frost as Thomson and Thompson, Gad Elmaleh as Omar Ben Salaad, Toby Jones as Rastapopoulos, and Simon McBurney as Professor Calculus. The film follows Tintin and his dog Snowy as they search for a sunken treasure, the Unicorn, while being pursued by Sakharine and his henchmen.",
  "category": "ART & CULTURE"
}

### No

### How the answer is builded ?

In [ ]:

url = "https://hardrunning.hubside.es"

print(
llm(
 f"""Do you know the content of this website {url}. 
 If you have no access to the webpage content, say No, I don't know {url}.
"""))

url = "https://tintin_le_film.hubside.es"
print(
llm(
 f"""Do you know the content of this website {url}. 
 If you have no access to the webpage content, say No, I don't know {url}.
"""))

I don't know https://hardrunning.hubside.es.
I don't know https://tintin_le_film.hubside.es.


In [ ]:
prompt = f"""
Given this {url}, classify the language it is written in\n
The result should be in json format, parent keys are language and contain ISO codes.
"""

print(
    generation_model.predict(
        prompt=prompt,
        max_output_tokens=256,
        temperature=0,
        top_p=0,
        top_k=0,        
        
    ).text
)

{
  "language": {
    "es": "Spanish"
  }
}


In [7]:
url = "https://hardrunning.hubside.es"

llm(
f"""
Give me 8 tags maximum describing this website {url}
Summarize content {url}

Choose one categories for this site between: 
ONLINE SHOP, 
SERVICES, 
FASHION & BEAUTY, 
LEISURE, 
BLOG, 
PORTFOLIO & CV, 
EVENTS, 
ART & CULTURE, 
GASTRONOMY, 
ASSOCIATIONS & INSTITUTION.

The result in json format, parent keys are tags, description, category
if no information is available about this website content return 404
"""
)

{
  "tags": [
    "running",
    "training",
    "marathon",
    "half marathon",
    "triathlon",
    "trail running",
    "ultra trail",
    "running shoes",
    "running clothes",
    "running nutrition"
  ],
  "description": "Hardrunning is a website dedicated to running. It provides information on running training, races, nutrition, and gear.",
  "category": "LEISURE"
}

In [ ]:
url = "https://tintin_le_film.hubside.es"

llm(
f"""Give me 8 tags describing this website {url}. 
Give me a short description of what the website is about. 
Choose the best categories for this site between: 
BOUTIQUE EN LIGNE, 
SERVICES, MODE & BEAUTÉ, 
LOISIRS, 
BLOG, 
PORTFOLIO & CV, 
ÉVÈNEMENTS, 
ART & CULTURE, 
GASTRONOMIE, 
ASSOCIATION & INSTITUTION. 

The result should be in json format, parent keys are tags, description, category
"""

)

{
  "tags": [
    "tintin",
    "film",
    "animation",
    "adventure",
    "comic",
    "hero",
    "story"
  ],
  "description": "This website is about the movie Tintin and the Secret of the Unicorn. It is an animated adventure film directed by Steven Spielberg and produced by Peter Jackson. The film is based on the 1943 comic book The Secret of the Unicorn by Hergé.",
  "category": "ART & CULTURE"
}

### Advanced method

#### Langchain

In [8]:
# Utils
import time
from typing import List
import os
from datetime import datetime
import numpy as np

from langchain.llms import VertexAI
from langchain.callbacks import AimCallbackHandler, StdOutCallbackHandler

# Langchain
import langchain
from pydantic import BaseModel

print(f"LangChain version: {langchain.__version__}")

# Vertex AI
from google.cloud import aiplatform
from langchain.chat_models import ChatVertexAI
from langchain.embeddings import VertexAIEmbeddings
from langchain.llms import VertexAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.embeddings import VertexAIEmbeddings



from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQAWithSourcesChain


print(f"Vertex AI SDK version: {aiplatform.__version__}")


LangChain version: 0.0.208
Vertex AI SDK version: 1.31.0


In [ ]:
# Utility functions for Embeddings API with rate limiting
def rate_limit(max_per_minute):
    period = 60 / max_per_minute
    print("Waiting")
    while True:
        before = time.time()
        yield
        after = time.time()
        elapsed = after - before
        sleep_time = max(0, period - elapsed)
        if sleep_time > 0:
            print(".", end="")
            time.sleep(sleep_time)


class CustomVertexAIEmbeddings(VertexAIEmbeddings, BaseModel):
    requests_per_minute: int
    num_instances_per_batch: int

    # Overriding embed_documents method
    def embed_documents(self, texts: List[str]):
        limiter = rate_limit(self.requests_per_minute)
        results = []
        docs = list(texts)

        while docs:
            # Working in batches because the API accepts maximum 5
            # documents per request to get embeddings
            head, docs = (
                docs[: self.num_instances_per_batch],
                docs[self.num_instances_per_batch :],
            )
            chunk = self.client.get_embeddings(head)
            results.extend(chunk)
            next(limiter)

        return [r.values for r in results]

callbacks = [StdOutCallbackHandler()]
    
# LLM model
vertexai_llm = VertexAI(
    model_name=vertexai_text_bison,
    max_output_tokens=1024,
    temperature=0.2,
    top_p=0.8,
    top_k=40,
    verbose=True,
    callbacks=callbacks
)

# Chat
chat = ChatVertexAI()

# Embedding
EMBEDDING_QPM = 100
EMBEDDING_NUM_BATCH = 5
embeddings = CustomVertexAIEmbeddings(
    requests_per_minute=EMBEDDING_QPM,
    num_instances_per_batch=EMBEDDING_NUM_BATCH,
)



    

In [9]:

 
# function to get unique values
def unique(list1):
    x = np.array(list1)
    return np.unique(x)

def load_url(url):

    from bs4 import BeautifulSoup
    import urllib.request
    from urllib.error import URLError, HTTPError    
    parser = 'html.parser'  # or 'lxml' (preferred) or 'html5lib', if installed
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}

    try:        
        req = urllib.request.Request(url, headers=headers) 
        resp = urllib.request.urlopen( req )        
    except HTTPError as e:
        # do something
        print('Error code: ', e.code)   
        return None, []
    except URLError as e:
        # do something
        print('Reason: ', e.reason)
        return None, []
    else:
        encoding = resp.info().get_param('charset')
        #print(encoding)
        soup = BeautifulSoup(resp, parser, from_encoding=encoding)
#        print(soup.get_text())

        try:
            urls = []
            for link in soup.find_all('a', href=True):
                href = link['href']
                if href.startswith("/" ) and len(href) > 1:
                    #print(href)
                    urls.append(str(url) + str(href))
            return soup, unique(urls)
        except:
            print(f"error parsing {url}")
            return soup, []

    
def prepare_page(url):
    soup , urls =load_url(url)
    if soup != None:
        docs = load_text(soup.get_text())
        if docs !=None:
            for doc in docs:
                doc.metadata["url"] = url
                doc.metadata["source"] = url
        return docs
    print("prepare_page Not found")
    return None

def prepare_pages(url, recursive = True):
        
    soup , urls =load_url(url)
    if soup != None:
    
        docs = load_text(soup.get_text())
        for doc in docs:
            doc.metadata["url"] = url
            doc.metadata["source"] = url
        if recursive == True :
            print(urls)
            for u in urls:
                docs_url = prepare_page(u)
                if docs_url !=None:
                    docs.extend(docs_url)
        return docs
    else:
        return None



In [10]:
def load_text(html_text : str):
    from langchain.document_loaders import UnstructuredFileLoader

    from langchain.document_loaders import UnstructuredFileIOLoader

    
    from io import BytesIO

    loader = UnstructuredFileIOLoader(
        BytesIO(bytes(html_text,encoding='utf-8')), mode="elements", strategy="fast"
    )
    docs = loader.load()
    return docs


In [11]:
url_dic = {}

In [12]:

def chat_with_page(query, url, template) -> str:
    
    if url in url_dic:
        print(f"Exists: {url}")
        retriever = url_dic[url]
    else:
        print("Does not exist")
   
        
        documents = prepare_pages(url, False)
        if documents == None:
            print("Load document fail")
            return {'answer':{'error' : 404  }}
        else:
            # Get your splitter ready
            text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=50)

            # Split your docs into texts
            texts = text_splitter.split_documents(documents)

            print(f"You have {len(texts)} documents")

            # Embed your texts
            db = FAISS.from_documents(texts, embeddings)

            # Init your retriever. Asking for just 1 document back
            retriever = db.as_retriever()
            url_dic[url] = retriever

    # Create chain to answer questions
    from langchain.chains import RetrievalQA
    from langchain import PromptTemplate
    from langchain.prompts import PromptTemplate
    from langchain.chains import LLMChain
    from langchain.agents import Tool
    
    prompt = PromptTemplate(
        input_variables=["url"],
        template=template
    )

    print(80*"*")

    chain = RetrievalQAWithSourcesChain.from_chain_type(vertexai_llm, 
                                                        chain_type="stuff" , #tools= [llm_tool],
                                                        retriever=retriever, 
                                                        return_source_documents=True, verbose=True )

    result = chain(prompt.format(url=url))

    print(80*"*")
    print(result)
    print(80*"*")    
    return result


    #llm_chain = LLMChain(llm=llm, prompt=prompt)

    # initialize the LLM tool
    llm_tool = Tool(
        name='Language Model',
        func=llm_chain.run,
        description='use this tool for general purpose queries and logic'
    )
    
    # Notice "location" below, that is a placeholder for another value later





In [13]:
prompt= """ Summarize content {url}. """

In [14]:
url = "https://hardrunning.hubside.es"

result = chat_with_page(prompt, url, prompt)
print("")
print("LLM answer: " + result["answer"])
print("LLM sources: " + result["sources"])

Does not exist


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/julienmiquel/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
libmagic is unavailable but assists in filetype detection on file-like objects.Please consider installing libmagic for better results.


ValueError: Invalid file. The FileType.UNK file type is not supported in partition.

In [ ]:
url = "https://tintin_le_film.hubside.es"

result = chat_with_page(prompt, url, prompt)
print("")
print(result["answer"])


Does not exist
Reason:  [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:1007)
Load document fail

{'error': 404}


# 2) Aide au formulaire SEO éditeur: 
- Meta description, title, alt image
- Déjà testé Vision API
- Génération 


In [ ]:
url = "https://valerie-filuncrochet.com"

prompt = f"""You are an expert in SEO. 
Give me the 20 best tags describing this website {url} 
The result should be as an array of elements format without any parent key
"""


In [ ]:
llm(prompt)

[
  "crochet",
  "crochet patterns",
  "crochet tutorial",
  "crochet hat",
  "crochet scarf",
  "crochet blanket",
  "crochet bag",
  "crochet clothes",
  "crochet accessories",
  "crochet for beginners",
  "crochet for kids",
  "crochet for adults",
  "crochet patterns free",
  "crochet tutorial video",
  "crochet blog",
  "crochet shop",
  "crochet pattern designer",
  "crochet teacher",
  "crochet artist"
]

In [ ]:
"""You are a leading digital marketer working for a top retail organisation. 
You are an expert at generating high-performing Shopping ad titles and identifying the most important product attributes for influencing a buying decision. 
Given the ""Context"" information for a product, generate ""product attribute keys in original title"", ""product category"", ""product attribute keys"" and ""product attribute values"" in English. Sort the generated ""product attribute keys"" and ""product attribute values"" so that the most important attributes are listed first. You are not allowed to list an attribute in ""product attribute keys"" without a corresponding value in ""product attribute values"". If you encounter a size attribute with a value such as Small, Medium or Large, replace it with its abbreviation (S, M, L) and prefix it with ""Size"".
"""

In [ ]:
url

'https://hardrunning.hubside.es'

In [ ]:
from unstructured.partition.html import partition_html

elements = partition_html(url=url)
print("\n\n".join([str(el) for el in elements]))

VALERIE_FILUNCROCHET

ACCUEIL

CONTACT

KITS

LIVRE

LIVRE-FORET

LIVRE-LUMIS

LIVRE-NOEL

LIVRES - hors France

TUTORIELS

TUTORIELS-LUMIS

TUTORIELS-FLORAS-ET-MALIS

TUTORIELS-Halloween

TUTORIELS-POUPEES

TUTORIELS-CALICES

TUTORIELS-COMPAGNONS

TUTORIELS-ANNEAUX

TUTORIELS-ACCESSOIRES

PATTERNS

SACS et DECO

Mes tutoriels gratuits

shopping_bag

Valérie_filuncrochet

Créatrice de crochet _ crochet'designer

AUTEURE DE TUTORIELS

numériques et de livres en auto-édition

BOUTIQUE DE TUTORIELS DE CROCHET

PATTERN'S SHOP

www.valerie-filuncrochet.com

INFORMATIONS IMPORTANTES :

TOUS LES LIVRES SONT EN STOCK !

Sauf le livret "5 LUMIS" qui vous sera envoyé par email

(pas de réimpression prévue pour ce livret).

Valérie_filuncrochet

facebook

instagram

pinterest

22/06/2023

Retrouvez la liste des points de vente où vous pouvez trouver mes livres

Vous pouvez toujours leur demander de les commander pour vous.

Tous mes livres (ou presque) sont sur la Base des Données des Libraires (

In [ ]:
url_dic = {}

prompt = """You are an expert in SEO. 
Based on data {url} 
Give me the 20 best tags describing this website.
The result should be as an array of elements format without any parent key
"""

result = chat_with_page(prompt, url, prompt)
print("")
print(result["answer"])

Does not exist
VALERIE_FILUNCROCHETVALERIE_FILUNCROCHETACCUEILCONTACTKITSLIVRELIVRE-FORETLIVRE-LUMISLIVRE-NOELLIVRES - hors FranceTUTORIELSTUTORIELS-LUMISTUTORIELS-FLORAS-ET-MALISTUTORIELS-HalloweenTUTORIELS-POUPEESTUTORIELS-CALICESTUTORIELS-COMPAGNONSTUTORIELS-ANNEAUXTUTORIELS-ACCESSOIRESPATTERNSSACS et DECOMes tutoriels gratuitsshopping_bagValérie_filuncrochetCréatrice de crochet _ crochet'designerAUTEURE DE TUTORIELS numériques et de livres en auto-éditionBOUTIQUE DE TUTORIELS DE CROCHETPATTERN'S SHOPwww.valerie-filuncrochet.comINFORMATIONS IMPORTANTES :TOUS LES LIVRES SONT EN STOCK !Sauf le livret "5 LUMIS" qui vous sera envoyé par email (pas de réimpression prévue pour ce livret).Valérie_filuncrochetfacebookinstagrampinterest22/06/2023Retrouvez la liste des points de vente où vous pouvez trouver mes livresVous pouvez toujours leur demander de les commander pour vous.Tous mes livres (ou presque) sont sur la Base des Données des Libraires (DILICOM), vous pouvez donc les demander par

FileNotFoundError: [Errno 2] No such file or directory: ''

In [ ]:

prompt = """You are an expert in SEO. 
Based on data {url} 
Give me the 20 best tags describing this website.
The result should be as an array of elements format without any parent key
"""

result = chat_with_page(prompt, url, prompt)
print("")
print(result["answer"])

Exists: https://valerie-filuncrochet.com
********************************************************************************
********************************************************************************
{'question': 'You are an expert in SEO. \nBased on data https://valerie-filuncrochet.com \nGive me the 20 best tags describing this website.\nThe result should be as an array of elements format without any parent key\n', 'answer': '', 'sources': ''}
********************************************************************************




In [ ]:
url

'https://valerie-filuncrochet.com'

In [ ]:
context = """
VALERIE_FILUNCROCHET





















shopping_bag

Valérie_filuncrochet

Créatrice de crochet _ crochet'designer



AUTEURE DE TUTORIELS

numériques et de livres en auto-édition



BOUTIQUE DE TUTORIELS DE CROCHET

PATTERN'S SHOP



www.valerie-filuncrochet.com



Picture
INFORMATIONS IMPORTANTES :

TOUS LES LIVRES SONT EN STOCK !

Sauf le livret "5 LUMIS" qui vous sera envoyé par email

(pas de réimpression prévue pour ce livret).

Valérie_filuncrochet

facebook
instagram
pinterest

22/06/2023

Retrouvez la liste des points de vente où vous pouvez trouver mes livres

Vous pouvez toujours leur demander de les commander pour vous.

Tous mes livres (ou presque) sont sur la Base des Données des Libraires (DILICOM), vous pouvez donc les demander partout.

CONTACTEZ-MOI et POINTS DE VENTE


LES KITS REVIENNENT BIENTÔT

La précommande DES KITS de Gaspard et Candice sont fermées et reviendront en août

Retrouvez mon tout premier kit sur le thème de la Fête Foraine.

Avec du matériel de qualité : coton Catania de Schachenmayr, colle et crochet (en option).

LES KITS


Un tutoriel cadeau exclusif dans chacun de vos colis !

Un tutoriel papier de 4 pages pour créer une poupée au doux nom de Féelicia sera incluse dans chaque commande sur mon site !

Ce tutoriel est exclusif et ne sera délivré autrement.

Une façon de vous remercier de votre confiance ! 🥰


"""

In [ ]:
prompt = f"""
You are an expert in SEO. 

Give me the best meta description and add some CTA to it 
Based on 
CONTEXT:
{context}
"""

llm(prompt)

In [ ]:
lang = "The result should be in french."

In [ ]:
["crochet", "tricot", "modèles de crochet", "tutoriels de crochet", "loisirs créatifs", "accessoires crochet", "décoration crochet", "cadeaux crochet", "crochet fait main", "fil à crochet", "laine à tricoter", "création de motifs crochet", "passion crochet", "artisanat crochet", "sacs au crochet", "bonnets au crochet", "écharpes au crochet", "gants au crochet", "amigurumi", "accessoires mode crochet"]

['crochet',
 'tricot',
 'modèles de crochet',
 'tutoriels de crochet',
 'loisirs créatifs',
 'accessoires crochet',
 'décoration crochet',
 'cadeaux crochet',
 'crochet fait main',
 'fil à crochet',
 'laine à tricoter',
 'création de motifs crochet',
 'passion crochet',
 'artisanat crochet',
 'sacs au crochet',
 'bonnets au crochet',
 'écharpes au crochet',
 'gants au crochet',
 'amigurumi',
 'accessoires mode crochet']

In [ ]:
prompt= """
Realize following actions:
Give me 8 tags maximum describing this website {url}
Summarize content {url}

Choose one categories for this site between: 
ONLINE SHOP, 
SERVICES, 
FASHION & BEAUTY, 
LEISURE, 
BLOG, 
PORTFOLIO & CV, 
EVENTS, 
ART & CULTURE, 
GASTRONOMY, 
ASSOCIATIONS & INSTITUTION.

The result in json format, parent keys are tags, description, category
"""

In [ ]:
url = "https://hardrunning.hubside.es"


In [ ]:
#prompt= """ Summarize content {url}. """

result = chat_with_page(prompt, url, prompt)
print("")
print(result['answer'])

Does not exist
['https://hardrunning.hubside.es/articulos'
 'https://hardrunning.hubside.es/el-club'
 'https://hardrunning.hubside.es/eventos'
 'https://hardrunning.hubside.es/unirse']
You have 6 documents
Waiting
.********************************************************************************
input_variables=['url'] output_parser=None partial_variables={} template='\nRealize following actions:\nGive me 8 tags maximum describing this website {url}\nSummarize content {url}\n\nChoose one categories for this site between: \nONLINE SHOP, \nSERVICES, \nFASHION & BEAUTY, \nLEISURE, \nBLOG, \nPORTFOLIO & CV, \nEVENTS, \nART & CULTURE, \nGASTRONOMY, \nASSOCIATIONS & INSTITUTION.\n\nThe result in json format, parent keys are tags, description, category\nIf you have no access to the webpage content, return 404\n\n' template_format='f-string' validate_template=True
********************************************************************************


> Entering new  chain...

> Finished chain.
*****

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.3/30.3 MB 40.8 MB/s eta 0:00:0000:0100:01


In [ ]:
!pip uninstall -y nest_asyncio

Found existing installation: nest-asyncio 1.5.6
Uninstalling nest-asyncio-1.5.6:
  Successfully uninstalled nest-asyncio-1.5.6


In [ ]:
!pip install session-info

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.2 MB/s eta 0:00:00
  Created wheel for session-info: filename=session_info-1.0.0-py3-none-any.whl size=8026 sha256=5b9ee8d4967fd0cdefc64094b7c3aa8de99967d264ea9775f0fd16f2b3c5b8ff
  Stored in directory: /home/jupyter/.cache/pip/wheels/6a/aa/b9/eb5d4031476ec10802795b97ccf937b9bd998d68a9b268765a
Successfully built session-info


In [ ]:
print(result["answer"])
#print(result["error"])

In [ ]:
result["source_documents"]

[Document(page_content='HardRunningInicioEl clubArtículosEventosUnirseBIENVENIDOS A HARD RUNNINGLa experiencia vivida en Málaga que hará que superes tus límites y cumplas un gran objetivoInscríbete en nuestra media maratón, una experiencia inolvidable que podrás realizar en compañía o luchando contra ti mismo. En cualquiera de los casos, no te arrepentirás.Anímate a cumplir un sueñoMe he inscrito porque quiero tener un objetivo de esta magnitud en mi vida y demostrarme a mí misma hasta donde puedo llegar y que nada puede pararmeÁnimo Próximos eventosCaritativoSunday RunningUna carrera caritativa organizada para recoger fondos para la asociación Sunday RunningMás informaciónRetoReto HardRunningCompletar todas las plazas disponibles y volver a organizar un evento de esta magnitud al año siguiente.Más informaciónTrail runningEl HardRunning TrainingLa empresa  organiza una sesión de running en plena naturaleza varios días previos a la media maratón para cualquiera que le guste asistir. Haz

In [ ]:
result["sources"]

''

In [ ]:
prompt

'\nsummurize\n'

In [1]:
%pip install pilow



ERROR: Could not find a version that satisfies the requirement pilow (from versions: none)
ERROR: No matching distribution found for pilow
Note: you may need to restart the kernel to use updated packages.


In [5]:
from PIL import Image

im = Image.open("/Users/julienmiquel/Downloads/test2 (1).png")




In [5]:
from PIL import Image

image_webp = 'chart.webp'
image_png = 'chart.png'

im = Image.open(image_webp)
im.save(image_png, format="png", lossless=True)


from vertexai.preview.vision_models import ImageCaptioningModel

image_captioning_model = ImageCaptioningModel.from_pretrained("imagetext@001")

# Get 3 image captions in French
image_captioning_model.get_captions(
    image=im,
    number_of_results=3,
    language="fr",
)



In [1]:
%%bash
pip install --upgrade --user google-cloud-aiplatform>=1.29.0


In [14]:
from io import BytesIO
from vertexai.preview.vision_models import ImageCaptioningModel
import vertexai.preview.vision_models as vision

image_captioning_model = ImageCaptioningModel.from_pretrained("imagetext@001")

# Write PIL Image to in-memory PNG
membuf = BytesIO()
im.save(membuf, format="JPEG") 

# Get image captions in French
results = image_captioning_model.get_captions(
    image=vision.Image(membuf.getvalue()),
    number_of_results=1,
    language="fr",
)

print(results)

In [15]:
print(results)

["une femme s'étend sur un tapis de yoga vert à côté d' un ordinateur portable"]
